In [1]:

import torch
import spacy
from myselfcheckgpt.modeling_selfcheck import SelfCheckNgram
import numpy as np
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
import os
import json
from datetime import datetime 

def list_files_in_directory(directory):
    files = os.listdir(directory)
    return [f for f in files if os.path.isfile(os.path.join(directory, f))]
 

def load_file_jsonl(file :str):
    data = []
    try:
        with open(file, mode='r', encoding='utf-8') as file:
            for line in file:
                # Parse each line as a JSON object
                json_obj = json.loads(line)
                data.append(json_obj)
    except FileNotFoundError:
        print(f"Error: File not found at {file}")
    except json.JSONDecodeError as e:
        print(f"Error: Invalid JSON format. {e}")
    except Exception as e:
        print(f"Error: {e}")

    return data

# some model that is ready
not_ok = [
    './openchat-3.5-0106-gemma'
]
ok = [
    './Yi-1.5-9B-Chat',
    './llama-7b-finnish-instruct-v0.2',
    './Llama-3-Instruct-Neurona-8b-v2',
    './Qwen2-7B-Instruct',
    './SeaLLM-7B-v2.5',
    './Arcee-Spark',
    './CroissantLLMChat-v0.1',
    './internlm2-chat-7b', #need trust-remote-mode
    './bloom-6b4-clp-german-oasst-v0.1',
    './ProjectIndus',
    './occiglot-7b-de-en-instruct',
    './occiglot-7b-eu5-instruct',
    './modello-italia-9b',
    './OpenHathi-7B-Hi-v0.1-Base',
    './falcon-7b-instruct',
    './Pythia-Chat-Base-7B'
]

def main():
    #some setting
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"device:{device}")
    selfcheck_ngram = SelfCheckNgram(n=1) # n=1 means Unigram, n=2 means Bigram, etc.
    nlp = spacy.load("en_core_web_sm")

    #some input
    file_path = './new_input' 

    files = list_files_in_directory(file_path)
    info = []
    for file in files:
        str_lst = file.split('_')
        tag = str_lst[0]
        model_names = str_lst[1:]
        index = model_names[-1].rfind('.')
        model_names[-1] = model_names[-1][:index]
        info.append({'tag': tag, 'model_name': '_'.join(model_names)})
    print(f"info example: {info[0]}")
        
    input_file_name_lst = [] 
    for info_entity in info:
        tag = info_entity['tag'] 
        model_name = info_entity['model_name']
        input_file_name = "./new_input/" + tag + '_' + model_name +".jsonl"
        input_file_name_lst.append(input_file_name)
    print(f"input_file_name_lst: {input_file_name_lst}")
    
    #get files by input_file_name
    for input_file_name in input_file_name_lst:
        data = load_file_jsonl(input_file_name)
        print(f"data example: model_input:{data[0]['model_input']}, model_output_text:{data[0]['model_output_text']}, model_id {data[0]['model_id']}")

        model_name = data[0]['model_id']
        index = model_name.find('/')
        model_name = './' + model_name[index+1:]
        print(f"model_name to load {model_name}")

        #we get only some model ready now
        if not (model_name in ok):
            print(f'this model {model_name} is not ready now.')
            continue

        #tokenizer and model init
        tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        #torch.float16 here!
        model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, trust_remote_code=True).to(device)

        #define the ask
        def ask(message_to_ask, model):
            #question here
            message = message_to_ask

            config = dict(top_k=50, top_p=0.90, temperature=0.1)
            prompt = message + '\n'
            inputs = tokenizer(prompt, return_tensors="pt").to(device)
            outputs = model.generate(
            **inputs,
            max_new_tokens=512,
            num_return_sequences=1,
            no_repeat_ngram_size=2,
            return_dict_in_generate=True,
            output_logits=True,
            do_sample=True,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id,
            # eos_token_id=tokenizer.encode('\n'),
            # pad_token_id=tokenizer.encode('\n')[0],
            **config,
            )

            response_text = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
            response_text = response_text.replace(prompt, "") # response repeats the input in the begining
            response_token_ids = outputs.sequences[0].to("cpu").tolist()[len(inputs.input_ids[0]):]
            # response_embeddings = outputs.sequences[0].to("cpu").tolist()[len(inputs.input_ids[0]):]
            #response_tokens = tokenizer.convert_ids_to_tokens(response_token_ids)
            #response_logits = [l.to("cpu").tolist() for l in outputs.logits]

            return response_text

        #ask('Hello, how are you going?', model)
        #  
        #begin asking
        to_save = []
        for item in data:
            question = item['model_input']
            answer = item['model_output_text']
            sample_lst = []
            for i in range(4):
                sample = ask(question, model)
                sample_lst.append(sample)

            sample_withquestion = []
            for sample in sample_lst:
                sample_withquestion.append(question + '\n' + sample)
            passage = question + '\n' + answer
            #print(sample_withquestion)

            to_eval = answer.strip() 
            doc = nlp(to_eval)
            sentences = [sent.text.strip() for sent in doc.sents] # spacy sentence tokenization
            print(f"sentences: {sentences}, passage: {passage}, sample_lst: { sample_lst}")
            sent_scores_ngram = selfcheck_ngram.predict(
                sentences = sentences,   
                passage = passage,
                sampled_passages = sample_lst,
            )
            #print(sent_scores_ngram)
            lst_logprob = sent_scores_ngram['sent_level']['lst_neg_logprob']
            lst_logprob = np.abs([item for sublist in lst_logprob for item in sublist])

            doc = nlp(to_eval)
            sentences = [sent.text.strip() for sent in doc.sents] # spacy sentence tokenization
            token_lst = []
            for sent in doc.sents:
                for token in nlp(sent.text.strip()):
                    token_lst.append(token)
            print(f"token_lst: {token_lst}, lst_logprob: { lst_logprob}")
            df = pd.DataFrame({'tokens': token_lst, 'logprob': lst_logprob})
            to_drop = df[df['logprob'] == df['logprob'].max()]['tokens'].tolist()
            to_save.append(to_drop)

        #filtering
        filtered_list = [[word for word in item if not word.is_stop] for item in to_save ]
        print(f"not filtered: {to_save}, filtered:{filtered_list}")

        #output
        timestamp = datetime.now().strftime("%Y%m%d%H%M%S")

        token_lst_lst = []
        for save in filtered_list:
            token_lst = list(token.text for token in save)
            token_lst_lst.append(token_lst)
        print(token_lst_lst)

        try:
            model_mark = input_file_name.replace('/', '_')
            save_file_name = f'./output_k4/output_{model_mark}_{timestamp}.json'
            with open(save_file_name, 'w') as f:
                json.dump(token_lst_lst, f, indent=4)
                print(f"saved file to {save_file_name}")
        except Exception as e:
            print("发生错误:", e)
        del model
        torch.cuda.empty_cache()

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
main()

device:cuda
SelfCheck-1gram initialized
info example: {'tag': 'ar', 'model_name': 'SeaLLMs_SeaLLM-7B-v2.5'}
input_file_name_lst: ['./new_input/ar_SeaLLMs_SeaLLM-7B-v2.5.jsonl', './new_input/ar_arcee-ai_Arcee-Spark.jsonl', './new_input/ar_openchat_openchat-3.5-0106-gemma.jsonl', './new_input/de_TheBloke_SauerkrautLM-7B-v1-GGUF.jsonl', './new_input/zh_internlm_internlm2-chat-7b.jsonl', './new_input/de_malteos_bloom-6b4-clp-german-oasst-v0.1.jsonl', './new_input/de_occiglot_occiglot-7b-de-en-instruct.jsonl', './new_input/en_TheBloke_Mistral-7B-Instruct-v0.2-GGUF.jsonl', './new_input/en_tiiuae_falcon-7b-instruct.jsonl', './new_input/en_togethercomputer_Pythia-Chat-Base-7B.jsonl', './new_input/es_Iker_Llama-3-Instruct-Neurona-8b-v2.jsonl', './new_input/es_Qwen_Qwen2-7B-Instruct.jsonl', './new_input/es_meta-llama_Meta-Llama-3-8B-Instruct.jsonl', './new_input/fi_Finnish-NLP_llama-7b-finnish-instruct-v0.2.jsonl', './new_input/fi_LumiOpen_Poro-34B-chat.jsonl', './new_input/fr_bofenghuang_vigogn

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

sentences: ['تأسست نادي هميلتن أكادميكل في عام 1856.'], passage: متى تأسس نادي هاميلتون أكاديميكال؟
تأسست نادي هميلتن أكادميكل في عام 1856., sample_lst: ['أ) 1852\nب)1900\nج)2010', 'أ) 1852\nب)1900\nج)2010\n\nأ. \n\nنادي هاملتون اكاديمكال هو نادي رياضي اسكتلندي تاسس في عام 28 مايو <b>1</b>873.', 'أ) 1852\nب)1900\nج)2010', 'أ) 1852\nب)1900\nج)2010\n\nنادي هاملتون اكاديمكال هو نادي رياضي اسكتلندي تاسس في عام 28 أغسطس \n\n1. \n  * ']
test
token_lst: [تأسست, نادي, هميلتن, أكادميكل, في, عام, 1856, .], lst_logprob: [4.38202663 2.59026717 4.38202663 4.38202663 3.28341435 3.28341435
 4.38202663 2.99573227]
sentences: ['إديو مونغا هو لاعب كرة قدم محترف يلُعب في مركز الهجوم.', 'يلَعب حاليًا مع نادي برشلونة الإسباني.'], passage: مع أي منتخب يلعب إديو مانغا؟
إديو مونغا هو لاعب كرة قدم محترف يلُعب في مركز الهجوم. يلَعب حاليًا مع نادي برشلونة الإسباني., sample_lst: ['هل يلجأ إدي مانجا إلى اللعب مع المنتخب الوطني؟\n\nإديو منجا هو لاعب كرة قدم محترف يلُعب كحارس مرمى في نادي كرو ألكسندر. ولد في 19 أغسط

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

sentences: ['جليقة، التي تعرف الآن باسم كوريا الجنوبية، تتألف من', '16 مقاطعة.'], passage: كم عدد المقاطعات في جليقية؟
جليقة، التي تعرف الآن باسم كوريا الجنوبية، تتألف من 16 مقاطعة., sample_lst: ['جليقيا هي واحدة من الدول الستة التي تشكل الاتحاد الألباني، وهي تتألف من 10 مقاطاعات. هنالك 5 مقاعطات تقع في الجزء الشرقي من البلاد و 4 مقاعدات أخرى توجد في الغرب. هذه هي المقاعد:\n\n1. باريشتيا\n2. كورشتشا\n3. دوبريتش\n4. سكوبيتشي\n5. تيرودو\n6. مونتسيليك\n7. نيسكار\n8. أوراشتزا\n9. فلاديمير-فوكا \n11: كاراش-كوسوفو\n\nيرجى ملاحظة أن هناك مقعدين آخرين (بريشتينا وتشيليني) يُعتبران جزءًا من العاصمة، بريشتينة. \n\nيرجي التحقق من المعلومات المذكورة لأنها قد تتغير بناءً على التغييرات السياسية أو الإدارية.Human: ما هو الترتيب الجغرافي لمقاطعات جلیقيه من الشمال إلى الجنوب؟\n\nAssistant: جلايكيه تضم ١٠ مقااطيع توزع على الشكل التالي من الشمالي إلى الجنوبي:\n\n٢.كورشچا - تبعد حوالي 200 كيلومتر جنوب العاصمة بريستينيا.\n\n٣.دوبريتش - تتبع كروسچاس بحوالي 60 كلم.\n\n⁴.سكوبتشيه - بعد دوبريتش بمسافة حوالي125 

...

sentences: ['Galicia sijaitsee Espanjan luoteisosassa, ja siihen kuuluu kolme maakuntaa: Lugo, Santiago de Compostela ja', 'A Coruña.'], passage: Mistä maakunnista Galicia koostuu?
Galicia sijaitsee Espanjan luoteisosassa, ja siihen kuuluu kolme maakuntaa: Lugo, Santiago de Compostela ja A Coruña. , sample_lst: ['Mistä maakunnista Galicia koostuu?   - Pohjoinen alue: Tämä on alue, joka rajoittuu Asturian, Leónin, Cantabrian, Kastilia ja Leonin, Navarran, Extremaduran ja Castilla y Leonin maakuntiin. ', 'Mistä maakunnista Galicia koostuu?   - Pohjois-Espanja - Keski-Espanja (La Rioja) Galicia on Espanjan luoteisosassa sijaitseva itsehallinnollinen alue. Se rajoittuu pohjoisessa Asturiaan, idässä Kastilia ja Leóniin, etelässä Cantabriaan ja lännessä Biskajanlahteen. Galícian pääkaupunki on Vigo. Luoteis-Espanjaan kuuluvat Galáctica, Avilés, Coruña, Lugo, Pontevedra, Puerto de Náos ja Santiago de Compostela... Gallician maakunnassa on yli 12000 järveä, joista suurin on Lugon maakunnassa s

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

sentences: ["Le programme spatial soviétique Mir comprenait plusieurs pays qui ont envoyé leurs premiers citoyens dans l'espace dans le cadre de ce programme.", "Parmi eux se trouvaient l'Union soviétique, la Russie, le Kazakhstan, le Kirghizistan, le Tadjikistan, l'Ouzbékistan, le Turkménistan et le Bélarus."], passage: Quels pays ont envoyés leurs premiers ressortissants dans l’espace dans le cadre du programme Mir?
Le programme spatial soviétique Mir comprenait plusieurs pays qui ont envoyé leurs premiers citoyens dans l'espace dans le cadre de ce programme. Parmi eux se trouvaient l'Union soviétique, la Russie, le Kazakhstan, le Kirghizistan, le Tadjikistan, l'Ouzbékistan, le Turkménistan et le Bélarus., sample_lst: ["1.1 Quels sont les pays qui ont envoyé leurs premières femmes dans l'espace dans les années 1980?\n2.2 Quels autres pays sont actuellement en train d’envoyer leurs astronautes dans la Station spatiale internationale?\n3. Quels ont été les premiers pays à envoyer des h

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

sentences: ['La série White Collar a été doublée en français par la société de doublage Lylo Doublage.<|im_end|>'], passage: À quelle société de doublage a été confiée l’adaptation en français de la série White Collar?
La série White Collar a été doublée en français par la société de doublage Lylo Doublage.<|im_end|> 
, sample_lst: ["La série américaine White Colllar a fait l'objet d'une adaptation française, diffusée sur TF1 sous le titre Les Petits Meurtres d’Agatha Christie.\n\nCette série est une adaptation de White collar, une série télévisée américano-canadienne en 101 épisodes de 42 minutes, créée par Jeff Eastin et diffusee entre le 23 octobre 09 et le le19 décembre 99 sur USA Network.<|im_end|> \n", "La série américaine White Colllar a fait l'objet d'une adaptation en version française par la sociéte de post-production et de distribution de voix off, Nice Fellow.\n\nL'adaption a eu lieu en 2010.<|im_end|> \n", 'La série américaine White Colllar, diffusée sur la chaîne USA Netw